## Imports

In [54]:
# needed imports
import numpy as np
import scipy.constants as sci_con
import matplotlib.pyplot as plt
import matplotlib
import os, sys
import dispersion as dis
from scipy import interpolate
from pathlib import Path
import helper_functions as helper
import vtk
import vtk.util.numpy_support as vnp

# matplotlib properties
plt.rc("font", family="sans-serif", size=12)

### Adjust Settings

In [55]:
# define type of texture and working dirs

texture = 'cos-'
texture_height = 3


# define settings for JCMsuite solver

convergence_variable = 'p' #switch this value between 'p' or 'h'
fem_degree = 7             #change this value if using p 
side_length = 0.5          #change this value if using h
precision = 1e-9


if convergence_variable == 'p':
    convergence_variable_value = fem_degree

elif convergence_variable == 'h':
    convergence_variable_value = side_length

# insert path to the root directory of JCMsuite and import
jcm_root = #insert root_dir

In [ ]:
sys.path.append(os.path.join(jcm_root, 'ThirdPartySupport', 'Python'))
import jcmwave as jcm

### Define Relavant Paths here

In [ ]:
# insert path to the working dirs 
working_dir = Path.cwd().parent
data_dir = os.path.join(working_dir, 'data', 'jcmsuite')
results_dir = f'{working_dir}/results/H_T={texture_height}/{convergence_variable}/{convergence_variable_value}'

for dir in [data_dir,results_dir]:
    if not os.path.exists(dir):
        os.makedirs(dir,exist_ok=True)

## Data Setup

In [56]:
# Get AM1.5 Spectrum
spectrum, spec_irradiance = helper.get_spectrum('thin')

spec_dist = spectrum[1:] - spectrum[:-1]
spec_dist = np.append(spec_dist,spec_dist[-1])*1e9

irradiance = spec_dist*spec_irradiance

# Set Angels of Incidence
thetas = [0]

In [57]:
# Set width and thickness of each layer
UNIT_OF_LENGTH = 1e-7
width = 7.5
glass_no_k_thickness = 1
glass_thickness = 10
ITO_thickness = 1.35
PTAA_thickness = 0.1
PVK_thickness = 4 - texture_height/2
C60_thickness = 0.3
Cu_thickness = 1

# Set heights from the ground up
Cu_height = 0
C60_height = Cu_height + Cu_thickness
PVK_height = C60_height + C60_thickness
PTAA_height = PVK_height + PVK_thickness
ITO_height   = PTAA_height + PTAA_thickness
glass_height = ITO_height + ITO_thickness
total_height = glass_no_k_thickness + glass_thickness + glass_height

# Set non mutable keyset for JCMsuite
jcm_keys ={
    'width'          : width,
    'glass_thickness': glass_thickness,
    'ITO_thickness'  : ITO_thickness,
    'PTAA_thickness' : PTAA_thickness,
    'PVK_thickness'  : PVK_thickness,
    'C_60_thickness' : C60_thickness,
    'Cu_thickness'   : Cu_thickness,
    'texture'        : texture,
    'texture_height' : texture_height
}

In [ ]:
# Prepare Material Data

# path to materials
material_path = os.path.join(working_dir, 'data')

# get Materials
#air       = dis.Material(file_path= material_path + '/Air.nk')
glass_no_k = dis.Material(file_path= material_path + '/glass_no_k.nk')
glass      = dis.Material(file_path= material_path + '/glass.nk')
ITO        = dis.Material(file_path= material_path + '/ITO.nk')
PTAA       = dis.Material(file_path= material_path + '/PTAA.nk')
PVK        = dis.Material(file_path= material_path + '/PVK.nk')
C_60       = dis.Material(file_path= material_path + '/C60.nk')
Cu         = dis.Material(file_path= material_path + '/Cu.nk')

# get permitivitys
permi_air        = 1 #air.get_permittivity()
permi_glass_no_k = glass_no_k.get_permittivity(spectrum)
permi_glass 	 = glass.get_permittivity(spectrum)
permi_ito 		 = ITO.get_permittivity(spectrum)
permi_PTAA       = PTAA.get_permittivity(spectrum)
permi_PVK 		 = PVK.get_permittivity(spectrum)
permi_C_60 		 = C_60.get_permittivity(spectrum)
permi_Cu 		 = Cu.get_permittivity(spectrum)

In [59]:
# glass Correction 
n_air  = np.real(permi_air)
n_glass = np.real(permi_glass_no_k)

R_0s = helper.fresnel_s(n_air, n_glass) 
R_0p = helper.fresnel_p(n_air, n_glass) 

In [60]:
# Define Computational domain
points_cd = [0, 0, 
             width, 0, 
             width, total_height, 
             0, total_height]

# define a sinusoidal profile
cos_n_periods = 1
cos_period = width
cos_amplitude = texture_height/2
cos_phase = np.pi

n_points = int(np.ceil(cos_n_periods*cos_period))
x = np.linspace(0, width, n_points)
y = cos_amplitude*np.cos(cos_phase + 2*np.pi*x/cos_period) + cos_amplitude
# make sure to be periodic
assert y[0] == y[-1]
# smooth profile
xx = np.linspace(0, width, n_points*4)
cos_spline = interpolate.CubicSpline(x, y)
yy = cos_spline(xx)


# texturize layers
glass_points = np.zeros(2*len(xx) + 4)
glass_points[0:2*len(xx):2] = xx
glass_points[1:2*len(xx):2] = glass_height + yy
glass_points[-4:] = [width, total_height, 
                    0, total_height]

ITO_points = np.copy(glass_points)
ITO_points[1:2*len(xx):2] -= ITO_thickness

PTAA_points = np.copy(ITO_points)
PTAA_points[1:2*len(xx):2] -= PTAA_thickness
    
PVK_points = np.array([0, PVK_height, 
                        width, PVK_height, 
                        width, total_height, 
                        0, total_height])

In [61]:
# Define the keyset for the parameter search

scan1,scan2 = np.meshgrid(spectrum,thetas)
dim1, dim2  = scan1.shape
keyset = np.array([[{} for _ in range(dim2)] for _ in range(dim1)])

for ii in np.ndindex(scan1.shape):

    keyset[ii]                     = dict(jcm_keys)
    keyset[ii]['wavelength']       = round(scan1[ii],10)
    keyset[ii]['theta'] 	       = scan2[ii]
    keyset[ii]['precision']        = precision
    keyset[ii]['fem_degree']       = fem_degree
    keyset[ii]['side_length']      = side_length
    keyset[ii]['width']            = width
    keyset[ii]['permi_glass_no_k'] = permi_glass_no_k[ii[1]]
    keyset[ii]['permi_glass']  = permi_glass[ii[1]]
    keyset[ii]['permi_ito']    = permi_ito[ii[1]]
    keyset[ii]['permi_PTAA']   = permi_PTAA[ii[1]]
    keyset[ii]['permi_PVK']    = permi_PVK[ii[1]]
    keyset[ii]['permi_C60']    = permi_C_60[ii[1]]
    keyset[ii]['permi_Cu']     = permi_Cu[ii[1]]
    keyset[ii]['points_cd']    = points_cd
    keyset[ii]['glass_points'] = glass_points
    keyset[ii]['PTAA_points']  = PTAA_points
    keyset[ii]['ITO_points']   = ITO_points
    keyset[ii]['PVK_points']   = PVK_points

In [ ]:
# Run JCMSuite for the defined setup on ZIB server
jcm.daemon.shutdown()
jcm.daemon.add_queue(Hostname = #add_hostname
                    Login = #add username
                    JCMROOT = jcm_root,
                    SSHClient = 'ssh -4',
                    PartitionName = #add Partition Name
                    Multiplicity = 16,
                    NThreads = 24,
                    Time = 10 * 60.,
                    WorkingDir = #add working_dir
                    MemoryPerJob = 64. * 1000)

job_ids = []

for ii in np.ndindex(scan1.shape):
    keys = keyset[ii]
    current_dir = f'{data_dir}/{round(keys['wavelength'],10)}'
    job_id = jcm.solve('project.jcmpt', working_dir = current_dir, keys=keys)
    job_ids.append(job_id)

jcm.daemon.wait(job_ids)

# Post Processes


In [63]:
### Computation of Photogeneration
## Initialize the arrays and objects by copying the size from a given file 

# Initalize the vtk reader
reader = vtk.vtkXMLRectilinearGridReader()
reader.SetFileName(f'{data_dir}/{round(spectrum[0],10)}/project_results/electric_field_absorption_density.vtr')
reader.Update()
# Read the vtk object
InputGrid = reader.GetOutput()

# Make a copy of the Object
OutputGrid = vtk.vtkRectilinearGrid()
OutputGrid.DeepCopy(InputGrid)
# Remove the existing Data
for i in range(4):
    OutputGrid.GetPointData().RemoveArray(0)
# Copy the shape of the Data
photo_generation = np.zeros_like(vnp.vtk_to_numpy(InputGrid.GetPointData().GetArray('ElectromagneticFieldAbsorptionDensity_xyz_1_real')))

# Get the Points of the underlying Grid ([[x1,y1,z1],[x2,y2,z2],...])
Points = vtk.vtkPoints()
OutputGrid.GetPoints(Points)
points = vnp.vtk_to_numpy(Points.GetData())

# Get Dimensions
x_dim, y_dim, z_dim = OutputGrid.GetDimensions()

In [ ]:
# Iterrate over all wavelengths
reader_1 = vtk.vtkXMLRectilinearGridReader()
reader_2 = vtk.vtkXMLRectilinearGridReader()

for i, wavelength in enumerate(spectrum):
    wavelength = round(wavelength,10)
    if i%10 == 0:
        print(f'Progress: {round(i/len(spectrum),2)}, Current WL: {wavelength}')

    # Read the XML file of the current wavelength
    reader_1.SetFileName(f'{data_dir}/{wavelength}/project_results/electric_field_absorption_density.vtr')
    reader_1.Update()
    EMFAD = reader_1.GetOutput()


    # Get the point data of the p-polarized and s-polarized light
    EMFAD_s = vnp.vtk_to_numpy(EMFAD.GetPointData().GetArray('ElectromagneticFieldAbsorptionDensity_xyz_1_real'))
    EMFAD_p = vnp.vtk_to_numpy(EMFAD.GetPointData().GetArray('ElectromagneticFieldAbsorptionDensity_xyz_2_real'))

    photo_generation += (EMFAD_s + EMFAD_p)/2 * irradiance[i] * wavelength/(sci_con.h * sci_con.c)

# Convert Numpy Data to vtk
PhotoGeneration = vnp.numpy_to_vtk(photo_generation)
PhotoGeneration.SetName('Photogeneration')

# Assign the Data to the OutputGrid
OutputGrid.GetPointData().AddArray(PhotoGeneration)

# Define and use the writer
writer = vtk.vtkXMLRectilinearGridWriter()
writer.SetInputData(OutputGrid)
writer.SetFileName(f'{results_dir}/PhotonOutput_{texture}.vtr')
writer.Write()

# Save data
np.save(f'{results_dir}/Photogeneration', photo_generation.reshape((x_dim,y_dim,z_dim),order='F'))


In [65]:
### Computation of Photo and Equvivalent Current

integrated_photo_generation = 0
integrated_parasitic_loss = 0
integrated_reflective_loss = 0 

for i, wavelength in enumerate(spectrum):
    wavelength = round(wavelength,10)

    integrated_absorption_density = jcm.loadtable(f'{data_dir}/{wavelength}/project_results/integrated_absorption.jcm')['ElectromagneticFieldAbsorption']
    integrated_photo_generation += (integrated_absorption_density[0] + integrated_absorption_density[1])/2 * irradiance[i] * wavelength/(sci_con.h * sci_con.c)

    if wavelength <= 8e-7:
        integrated_parasitic_absorption = jcm.loadtable(f'{data_dir}/{wavelength}/project_results/integrated_parasitic_absorption.jcm')['ElectromagneticFieldAbsorption']
        integrated_parasitic_loss += (integrated_parasitic_absorption[0] + integrated_parasitic_absorption[1])/2 * irradiance[i] * wavelength/(sci_con.h * sci_con.c)

    if wavelength <= 8e-7:
        reflection_data = jcm.convert2powerflux(jcm.loadtable(f'{data_dir}/{wavelength}/project_results/reflection.jcm')) 
        integrated_reflective_loss += (np.abs(np.sum(reflection_data['PowerFluxDensity'][0][:,1])) + np.abs(np.sum(reflection_data['PowerFluxDensity'][1][:,1])))/2 * irradiance[i] * wavelength/(sci_con.h * sci_con.c)

max_photo_current = integrated_photo_generation*sci_con.e/(width*UNIT_OF_LENGTH)
max_photo_current /= 10 #mA + cm^-2

parsitic_equivalent_current = integrated_parasitic_loss*sci_con.e/(width*UNIT_OF_LENGTH*10)
reflective_equivalent_current = integrated_reflective_loss*sci_con.e/10

np.save(f'{results_dir}/J_gen', max_photo_current)
np.save(f'{results_dir}/J_par', parsitic_equivalent_current)
np.save(f'{results_dir}/J_R', reflective_equivalent_current)


## Some Plots


In [ ]:

rel_avg_error = np.array([])
rel_max_error = np.array([])
rel_error_J   = np.array([])

if convergence_variable == 'p':
    fine_value = 9
    params = sorted(os.listdir(f'{working_dir}/results/H_T={texture_height}/{convergence_variable}'))[:-1]
else:
    fine_value = 0.0625
    params = sorted(os.listdir(f'{working_dir}/results/H_T={texture_height}/{convergence_variable}'))[1:]
    params.reverse()

fine_photo_generation = np.load(f'{working_dir}/results/H_T={texture_height}/{convergence_variable}/{fine_value}/Photogeneration.npy')
fine_current = np.load(f'{working_dir}/results/H_T={texture_height}/{convergence_variable}/{fine_value}/Max_Photo_Current.npy')
fine_photo_generation[fine_photo_generation <= 1e10] = 0

for param in params:

    rought_results_dir = f'{working_dir}/results/H_T={texture_height}/{convergence_variable}/{param}'
    rough_photo_generation = np.load(f'{rought_results_dir}/Photogeneration.npy')
    rough_photo_generation[rough_photo_generation<=1e10] = 0
    rough_current = np.load(f'{rought_results_dir}/Max_Photo_Current.npy')
    
    rel_error     = np.divide(abs(fine_photo_generation - rough_photo_generation),fine_photo_generation, where= fine_photo_generation!=0)
    rel_max_error = np.append(rel_max_error, np.amax(rel_error))
    rel_avg_error = np.append(rel_avg_error, np.average(rel_error))
    rel_error_J   = np.append(rel_error_J, np.divide(abs(fine_current - rough_current), fine_current, where= fine_current!=0))  



plt.figure(dpi=300,figsize=(40,40))

color = 'tab:red'
fig, ax1 = plt.subplots()
plt.title("Numerical Accuracy of Photogeneration")
ax1.set_xlabel('FEM Degree', fontsize=20)
ax1.set_ylabel('Avg. Rel. Error', color=color, fontsize=20)
ax1.set_yscale('log')
ax1.plot(params, rel_avg_error, marker='H',color=color)
ax1.tick_params(axis='y', labelcolor=color)

    

ax2 = ax1.twinx()  # instantiate a second Axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Max Rel. Error', color=color, fontsize=20)  # we already handled the x-label with ax1
ax2.set_yscale('log')
ax2.plot(params, rel_max_error, marker='.', linestyle='--', color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped

plt.savefig(f'{working_dir}/results/H_T={texture_height}/Numerical_Error_{convergence_variable}_{texture_height}.pdf')
plt.show()



plt.figure(dpi=300,figsize=(40,40))
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20)

fig, ax1 = plt.subplots()

plt.title("Numerical Accuracy of ${J}_{gen}$")

color = 'tab:green'
ax1.set_xlabel('FEM Degree', fontsize=20)
ax1.set_ylabel('Rel. Error',  color = color, fontsize=20)
ax1.plot(params, rel_error_J, marker='H', color = color)
ax1.tick_params(axis='y', labelcolor = color)
ax1.set_yscale('log')


fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.savefig(f'{working_dir}/results/H_T={texture_height}/Numerical_Accuracy_J_gen_{convergence_variable}_{texture_height}.pdf')
plt.show()